# IMPORT

In [1]:
import os
from pdf2image import convert_from_path
import ntpath
from google.oauth2 import service_account
from google.cloud import vision
import io
import string
import pandas as pd
import ast
import spacy
import shutil
from shutil import copyfile
nlp = spacy.load('en_core_web_sm')

In [2]:
from __future__ import unicode_literals, print_function
from spacy.lang.en import English # updated

# PDF TO TEXT

In [3]:
def convert_pdf_2_image(uploaded_image_path, uploaded_image):
    project_dir = os.getcwd()
    os.chdir(uploaded_image_path)
    file_name = str(uploaded_image).replace('.pdf','')
    pages = convert_from_path(uploaded_image, 200,poppler_path='/Users/kunal/Documents/VdartWorking/Poppler/poppler-0.68.0_x86/poppler-0.68.0/bin/')
    pageNumCount = 1
    outputNames = []
    for page in pages:
        output_file = file_name+"_"+str(pageNumCount) + '.jpg'
        page.save(output_file, 'JPEG')
        pageNumCount +=1
        outputNames.append(output_file)
    return outputNames

In [4]:
def convert_img_2_text(imagePath):
    keyDIR = "/Users/kunal/Documents/ResumeNLPVdart/APIKEYSGOOGLE/resumeMatcher-pdf2img.json"
    credentials = service_account.Credentials.from_service_account_file(keyDIR)
    client = vision.ImageAnnotatorClient(credentials=credentials)
    with io.open(imagePath, 'rb') as image_file:
        content = image_file.read()
    image = vision.types.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    totalString = ''
    for text in texts:
        totalString+=text.description
    totalString = totalString.rsplit(' ', 1)[0]
    return totalString

In [5]:
def deleteIMG(path):
    for i in os.listdir(path):
        if i.endswith(".jpg"):
            os.remove(folder_pdf + i)
def deleteEverythingInFolder(folder_pdf):
    for file in os.listdir(folder_pdf):
        try:
            shutil.rmtree(folder_pdf+ file)
        except NotADirectoryError:
            try:
                os.remove(folder_pdf+ file)
            except:
                pass
def moveRenameAllFiles(sourceFolderResumes, folder_pdf):
    for realFile in os.listdir(sourceFolderResumes):
        copyfile(sourceFolderResumes + realFile, folder_pdf + realFile)
    os.chdir(folder_pdf)
    documentcounter = 100
    for i in os.listdir(folder_pdf):
        fileNameCreator = "Document_" + str(documentcounter) + ".pdf"
        print(i + "   changed to  " + fileNameCreator)
        os.rename(i, fileNameCreator) 
        documentcounter+=1

In [6]:
folder_pdf = '/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/'
sourceFolderResumes = '/Users/kunal/Documents/ResumeNLPVdart/50_resumes/'

In [7]:
deleteEverythingInFolder(folder_pdf)
moveRenameAllFiles(sourceFolderResumes, folder_pdf)
numberOfFiles = len([name for name in os.listdir(folder_pdf) if os.path.isfile(name)])

Document_402.pdf   changed to  Document_100.pdf
Document_403.pdf   changed to  Document_101.pdf
Document_405.pdf   changed to  Document_102.pdf
Document_406.pdf   changed to  Document_103.pdf
Document_407.pdf   changed to  Document_104.pdf
Document_408.pdf   changed to  Document_105.pdf
Document_409.pdf   changed to  Document_106.pdf
Document_410.pdf   changed to  Document_107.pdf
Document_411.pdf   changed to  Document_108.pdf
Document_412.pdf   changed to  Document_109.pdf
Document_413.pdf   changed to  Document_110.pdf
Document_414.pdf   changed to  Document_111.pdf
Document_415.pdf   changed to  Document_112.pdf
Document_417.pdf   changed to  Document_113.pdf
Document_418.pdf   changed to  Document_114.pdf
Document_419.pdf   changed to  Document_115.pdf
Document_420.pdf   changed to  Document_116.pdf
Document_421.pdf   changed to  Document_117.pdf
Document_422.pdf   changed to  Document_118.pdf
Document_423.pdf   changed to  Document_119.pdf
Document_424.pdf   changed to  Document_

In [8]:
os.mkdir("/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/textFiles")

In [9]:
textList = []
numberPagesList = []
textFile_dir = "/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/textFiles"
for i in os.listdir(folder_pdf):
    os.chdir(folder_pdf)
    folderNameCreate = folder_pdf+"Fold_"+i[:-4]+"/"
    print(folderNameCreate)
    os.mkdir(folderNameCreate)
    shutil.move(i,folderNameCreate)
    os.chdir(folderNameCreate)
    for j in os.listdir(folderNameCreate):
        print(j)
        if j.endswith(".pdf"):
            imgName = convert_pdf_2_image(folderNameCreate, j)
            print("Running Number :" + str(len(imgName))) 
            printString = ""
            for docName in imgName:
                printString += docName + " " 
            print(printString)
        for q in range(len(imgName)):
            text = convert_img_2_text(folderNameCreate + i[:-4] + "_" + str(q+1) + ".jpg")
            if not text == "": 
                if not "Recommendation" in str(text): 
                    os.chdir("/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/textFiles")
                    text_file_name = textFile_dir + "/"+ i[:-4] + "_" + str(q+1) + ".txt"
                    with io.open(text_file_name, "w", encoding="utf-8") as f:
                        f.write(text)
                    os.chdir("/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData")
                else:
                    print(text)
                    checkIfRecommendation = input("Is there a recommendation in this? -- Y or N")
                    if checkIfRecommendation == "N":
                        print("Running the Document")
                        os.chdir("/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/textFiles")
                        text_file_name = textFile_dir + "/"+ i[:-4] + "_" + str(q+1) + ".txt"
                        with io.open(text_file_name, "w", encoding="utf-8") as f:
                            f.write(text)
                        os.chdir("/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData")
            textList.append(text)    
            numberPagesList.append(len(imgName))    

/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_Document_100/
Document_100.pdf
Running Number :1
Document_100_1.jpg 
/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_Document_101/
Document_101.pdf
Running Number :1
Document_101_1.jpg 
/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_Document_102/
Document_102.pdf
Running Number :6
Document_102_1.jpg Document_102_2.jpg Document_102_3.jpg Document_102_4.jpg Document_102_5.jpg Document_102_6.jpg 
/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_Document_103/
Document_103.pdf
Running Number :5
Document_103_1.jpg Document_103_2.jpg Document_103_3.jpg Document_103_4.jpg Document_103_5.jpg 
/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_Document_104/
Document_104.pdf
Running Number :3
Document_104_1.jpg Document_104_2.jpg Document_104_3.jpg 
/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_Document_105/
Document_105.pdf
Running Number :2
Document_105_1.jpg Docume

/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_Document_130/
Document_130.pdf
Running Number :13
Document_130_1.jpg Document_130_2.jpg Document_130_3.jpg Document_130_4.jpg Document_130_5.jpg Document_130_6.jpg Document_130_7.jpg Document_130_8.jpg Document_130_9.jpg Document_130_10.jpg Document_130_11.jpg Document_130_12.jpg Document_130_13.jpg 
/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_Document_131/
Document_131.pdf
Running Number :11
Document_131_1.jpg Document_131_2.jpg Document_131_3.jpg Document_131_4.jpg Document_131_5.jpg Document_131_6.jpg Document_131_7.jpg Document_131_8.jpg Document_131_9.jpg Document_131_10.jpg Document_131_11.jpg 
/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_Document_132/
Document_132.pdf
Running Number :3
Document_132_1.jpg Document_132_2.jpg Document_132_3.jpg 
/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_Document_133/
Document_133.pdf
Running Number :3
Document_133_1.jpg Document_133_

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/Fold_textF/textF_1.jpg'

In [ ]:
text

## Convert to Excel

In [10]:
textFile_dir = "/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/textFiles"

In [11]:
fileName = []
for textFile in os.listdir(textFile_dir):
    fileName.append(["gs://input_resumes_pdf/textFiles/"+textFile])

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData/textFiles'

In [12]:
os.chdir("/Users/kunal/Documents/ResumeNLPVdart/GoogleLabelingData")
df = pd.DataFrame(fileName)
EXCELFILENAMESent = "textFilePaths.csv"
pd.DataFrame(df).to_csv(EXCELFILENAMESent)

# Test Sentiment

In [15]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.oauth2 import service_account
from google.cloud import language_v1
from google.cloud.language_v1 import enums

In [21]:
x =[4,5,3]
y = [3.3,5]

In [22]:
x.extend(y)

In [23]:
x

[4, 5, 3, 3.3, 5]